In [17]:
%%time
def move_head(arr, index: int):
    arr = arr = [arr[index]] + arr[:index] + arr[index + 1:]

def similarity(arr1, arr2):
    counter = 0
    total = max(len(arr1), len(arr2))
    for str1 in arr1:
        for str2 in arr2:
            if str1 in str2 or str2 in str1:
                counter += 1
    return counter / total * 100

def data_filter(data):
    filtered_data = []
    for w in data:
        word = ""
        for c in w:
            if not c.isdigit():
                word += c
                
        word = word.strip()
        
        if len(word) > 3 and word not in filtered_data:
            filtered_data.append(word)
    return filtered_data


import pandas as pd


good_addresses = []
count_addresses = 0
for x in range(0, 134, 10):
    addresses = pd.read_table(f"./data/{x}/ADDRESSES.csv", sep=',')
    
    for index, row in addresses.iterrows():
        a = row['address']
        count_addresses += 1
        data = data_filter(str(row['name']).split(",") + str(row['address']).split(","))
        flag = True
        for index, ga in enumerate(good_addresses):
            if similarity(ga, data) > 10:
                move_head(good_addresses, index)
                flag = False
                break
        if flag:
            good_addresses.append(data)
            # print(index, "ADD", data)
            
print(good_addresses)

print(len(good_addresses), count_addresses)
    
    

# 5676 136401


[['faculty of teacher education and sport', 'sogn og fjordane university collegesogndal', 'norway'], ['oslo university hospital', 'national centre for epilepsy', 'division for clinical neuroscience', 'section for adult epileptology', 'department of neurology', 'p.o. box', 'baerum postterminal', 'g.f. henriksens vei', 'sandvika', 'oslo', 'norway'], ['department of sports science and clinical biomechanics', 'university of southern denmark', 'campusvej', 'odense m', 'denmark; bergen university college', 'institute of occupational therapy', 'physiotherapy and radiography', 'department of health sciences', 'bergen', 'norway'], ['department of gynecology and obstetrics', 'horsens regional hospital', 'horsens', 'denmark department of research', 'denmark department of health science and technology', 'aalborg university', 'aalborg', 'denmark danish center of systematic reviews: a joanna briggs institute centre of excellence', 'the center of clinical guidelines - clearing house', 'denmark facult

In [22]:
%%time
import re

def data_filter(name: str, address: str):
    data = name.split(",") + address.split(",")
    filtered_data = []
    for word in data:
        cleaned_word = word.strip().lower()
        cleaned_word = re.sub(r'[^a-zA-Z\s]', ' ', cleaned_word)
        cleaned_word = re.sub(r'\s+', ' ', cleaned_word)
        if len(cleaned_word) > 2:
            filtered_data.append(cleaned_word)
    return ",".join(filtered_data)
            

def similarity(data1: str, data2: str):
    arr1, arr2 = data1.split("'"), data2.split("'")
    counter = 0
    total = max(len(arr1), len(arr2))
    for str1 in arr1:
        for str2 in arr2:
            if str1 in str2 or str2 in str1:
                counter += 1
    return counter / total * 100

import pandas as pd
import os

addresses = []
good_addresses = pd.DataFrame(columns=["clear", "index"])

for x in range(0, 134, 10):
    data = pd.read_table(f"./data/{x}/ADDRESSES.csv", sep=',')
    addresses.append(data)

df = pd.concat(addresses, ignore_index=True)
df = df.assign(clear=lambda x: data_filter(str(x["name"]), str(x["address"])))

for index, address in df.iterrows():
    flag = True
    for index_2, ga in good_addresses.iterrows():
        if similarity(ga["clear"], address["clear"]) > 10:
            # print(ga["clear"], address["clear"])
            flag = False
            break
    if flag:    
        good_addresses.loc[-1] = [address["clear"], index]
        good_addresses.index += 1

good_addresses["clear"]


CPU times: user 2.25 s, sys: 0 ns, total: 2.25 s
Wall time: 2.25 s


0     faculty of teacher education and sport centre...
Name: clear, dtype: object

In [ ]:
import os
import pandas as pd

dfs = []

for dirpath, dirnames, filenames in os.walk("./data/"):
    if dirnames:
        continue
    addresses = pd.read_table(os.path.join(dirpath, "ADDRESSES.csv"), sep=',')
    addresses_people = pd.read_table(os.path.join(dirpath, "ADDRESSES_PEOPLE.csv"), sep=',')
    people = pd.read_table(os.path.join(dirpath, "PEOPLE.csv"), sep=",")
    people_publications = pd.read_table(os.path.join(dirpath, "PEOPLE_PUBLICATIONS.csv"), sep=",")
    publications = pd.read_table(os.path.join(dirpath, "PUBLICATIONS.csv"), sep=",")

    addresses.rename(columns={"temp_id": "address_uuid"}, inplace=True)
    people.rename(columns={"temp_id": "person_uuid"}, inplace=True)
    publications.rename(columns={"temp_id": "publication_uuid"}, inplace=True)
    
    merged_1 = pd.merge(addresses, addresses_people, on='address_uuid', how="inner")
    merged_2 = pd.merge(merged_1, people, on="person_uuid", how="inner")
    merged_3 = pd.merge(merged_2, people_publications, on="person_uuid", how="inner")
    data = pd.merge(merged_3, publications, on="publication_uuid", how="inner")

    dfs.append(data)

df = pd.concat(dfs, ignore_index=True)

df.head()

In [ ]:
df['id'] = df['address_uuid'].astype(str) + "_" + df['person_uuid'].astype(str) + "_" + df['publication_uuid'].astype(str)
df = df[['id'] + [col for col in df.columns if col != 'id']]
df.drop(columns=['address_uuid', 'person_uuid', "publication_uuid"], inplace=True)

In [ ]:
df.groupby('address').size().reset_index(name='count').sort_values(by='count', ascending=False)

In [ ]:
visited = set()
c = 0
i = 0
for index, row in df.iterrows():
    i+=1
    current_address = row['address']
    current_name = row['name']
    v = str(current_address) + ", " + str(current_name)
    if v in visited:
        c += 1
        continue
    visited.add(v)

c, i, c-i
    
    

In [ ]:
df.at[1, "firstname"] = "Paweł"
df.iloc[1]